In [167]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [168]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [169]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [170]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [171]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [172]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [173]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [174]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [175]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1397
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [176]:
# Define election dates
election_date = datetime.strptime('2017-06-08', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=60)
prediction_date = election_date - timedelta(days=30)

In [177]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [178]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [179]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [180]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [181]:
# Define our X by dropping irrelevant and y columns

X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [182]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [183]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2017-06-08',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [184]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                             0.059484
months_to_elec_weight                  0.768621
rating                                 0.822511
party_in_power_Conservative                 1.0
party_in_power_Conservative_Liberal         0.0
poll_length                            2.121212
CON_FC                                 0.460909
LAB_FC                                 0.275455
LIB_FC                                 0.100303
BRX_FC                                      NaN
GRE_FC                                 0.031515
OTH_FC                                 0.014286
PLC_FC                                      NaN
SNP_FC                                 0.041923
UKI_FC                                 0.074545
dtype: object

In [185]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': 0, # doesnt exist in 2017
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT':    0.006,
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': averages['UKI_FC']
}

In [186]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_272786/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [187]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [188]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [189]:
actuals_2017 = {
    "CON": 0.4299,  # Conservative Party
    "LAB": 0.4013,  # Labour Party
    "LIB": 0.0737,  # Liberal Democrats
    "BRX": 0.0000,  # Brexit Party (did not exist in 2017)
    "GRE": 0.0170,  # Green Party
    "SNP": 0.0304,  # Scottish National Party
    "UKI": 0.0183,  # UK Independence Party
    "PLC": 0.0051,  # Plaid Cymru
    "OTH": 0.0243   # Other parties
}

In [190]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [191]:
# Print mean predictions
for party in parties:
    print(f"2017 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2017 actual result for {party}  : {actuals_2017[party]}")


2017 mean prediction for CON: 0.4609180986881256
2017 actual result for CON  : 0.4299
2017 mean prediction for LAB: 0.2758376896381378
2017 actual result for LAB  : 0.4013
2017 mean prediction for LIB: 0.10040300339460373
2017 actual result for LIB  : 0.0737
2017 mean prediction for BRX: 0.0
2017 actual result for BRX  : 0.0
2017 mean prediction for GRE: 0.03151073306798935
2017 actual result for GRE  : 0.017
2017 mean prediction for SNP: 0.041924167424440384
2017 actual result for SNP  : 0.0304
2017 mean prediction for UKI: 0.07458362728357315
2017 actual result for UKI  : 0.0183
2017 mean prediction for PLC: 0.005920048337429762
2017 actual result for PLC  : 0.0051
2017 mean prediction for OTH: 0.014399328269064426
2017 actual result for OTH  : 0.0243


In [192]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [193]:
# Print mean of predictions
for party in parties:
    print(f"2017 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2017 actual result for {party}  : {actuals_2017[party]}")


2017 last prediction for CON: 0.4607197344303131
2017 actual result for CON  : 0.4299
2017 last prediction for LAB: 0.27629873156547546
2017 actual result for LAB  : 0.4013
2017 last prediction for LIB: 0.10043760389089584
2017 actual result for LIB  : 0.0737
2017 last prediction for BRX: 0.0
2017 actual result for BRX  : 0.0
2017 last prediction for GRE: 0.03151073306798935
2017 actual result for GRE  : 0.017
2017 last prediction for SNP: 0.041924167424440384
2017 actual result for SNP  : 0.0304
2017 last prediction for UKI: 0.07450564205646515
2017 actual result for UKI  : 0.0183
2017 last prediction for PLC: 0.005920048337429762
2017 actual result for PLC  : 0.0051
2017 last prediction for OTH: 0.014404946938157082
2017 actual result for OTH  : 0.0243
